In [1]:
import numpy as np

In [2]:
import time

In [3]:
from scipy import optimize

# Root Finding Methods

Given a nonlinear function $f$, we seek a value $x$ for which $f(x)=0$.

## Bisection Method

The bisection method begins with an initial bracket and successively reduces its length until the solution has been isolated as accurately as desired. At each iteration, the function is evaluated at the midpoint of the current interval, and half of the interval can then be discarded, depending on the sign of the function at the midpoint.

In [17]:
def bisection(func,lower,upper,tol=1e-5,func_args=None):

    f_lower = func(lower,*(func_args or ()))
    f_upper = func(upper,*(func_args or ()))

    calls = 2

    while (upper-lower)>tol:

        middle = (lower+upper)/2

        f_middle = func(middle,*(func_args or ()))

        print(f"{calls:2} {lower:8.6f},{f_lower:8.6f},{upper:8.6f},{f_upper:8.6f}")

        calls += 1

        if f_middle*f_lower<0:
            upper,f_upper = middle,f_middle
        elif f_middle*f_upper<0:
            lower,f_lower = middle,f_middle
        else:
            info = "Either there is no root or even number of roots."
            middle,f_middle = float('nan'),float('nan')
            break
    else:

        info = f"Found a root after {calls} iterations."

    return middle, f_middle

In [18]:
def f(x):
    return x**2-4*np.sin(x)

In [21]:
bisection(f,1,3,tol=1e-7)

 2 1.000000,-2.365884,3.000000,8.435520
 3 1.000000,-2.365884,2.000000,0.362810
 4 1.500000,-1.739980,2.000000,0.362810
 5 1.750000,-0.873444,2.000000,0.362810
 6 1.875000,-0.300718,2.000000,0.362810
 7 1.875000,-0.300718,1.937500,0.019849
 8 1.906250,-0.143255,1.937500,0.019849
 9 1.921875,-0.062406,1.937500,0.019849
10 1.929688,-0.021454,1.937500,0.019849
11 1.933594,-0.000846,1.937500,0.019849
12 1.933594,-0.000846,1.935547,0.009491
13 1.933594,-0.000846,1.934570,0.004320
14 1.933594,-0.000846,1.934082,0.001736
15 1.933594,-0.000846,1.933838,0.000445
16 1.933716,-0.000201,1.933838,0.000445
17 1.933716,-0.000201,1.933777,0.000122
18 1.933746,-0.000039,1.933777,0.000122
19 1.933746,-0.000039,1.933762,0.000041
20 1.933746,-0.000039,1.933754,0.000001
21 1.933750,-0.000019,1.933754,0.000001
22 1.933752,-0.000009,1.933754,0.000001
23 1.933753,-0.000004,1.933754,0.000001
24 1.933753,-0.000001,1.933754,0.000001
25 1.933754,-0.000000,1.933754,0.000001
26 1.933754,-0.000000,1.933754,0.000000


(1.933753788471222, np.float64(1.3559806477658753e-07))

## Fixed Point Iteration

A nonlinear equation can often be recast as a $x=g(x)$ for a related nonlinear function $f(x)=0$, and the solution is based on iteration schemes of the form $x_{k+1}=g(x_k)$. Here, $g$ is a **suitably** chosen function whose fixed points are solutions for $f(x)=0$. Note that $x=g(x)$ is a fixed point of the function $g$, since $x$ is unchanged when $g$ is applied to it.

In [29]:
def fixed(f,g,x0,tol=1e-6,maxiter=100):

    xn = x0
    
    for i in range(maxiter):
        xn = g(xn)
    
        if abs(f(xn))<tol:
            break
    
        print(i,xn,f(xn))
    else:
        print("Could not converge to a solution")

For the nonlinear equation:

In [30]:
def f(x):
  return x**2-x-2

any of the choices

In [31]:
g1 = lambda x: x**2-2

In [32]:
g2 = lambda x: (x+2)**(1/2)

In [33]:
g3 = lambda x: 1+2/x

In [34]:
g4 = lambda x: (x**2+2)/(2*x-1)

In [39]:
fixed(f,g4,0.6,tol=1e-6,maxiter=100)

0 11.800000000000002 125.44000000000007
1 6.2495575221238955 30.807411700211468
2 3.5704459920916536 7.177638590351698
3 2.401619279483995 1.3661558841052277
4 2.042410709492021 0.1290307967556794
5 2.0005830704716154 0.0017495513860210643


## Newton's Method

More rapidly convergent methods can be derived by using the function values to obtain a more accurate approximation to the solution at each iteration.

In [50]:
def newton(func,prime,x0,tol=1e-5,maxiter=100,func_args=None,prime_args=None):
    
    calls = 0

    f = func(x0,*(func_args or ()))
    p = prime(x0,*(prime_args or ()))

    calls += 1

    xn = x0
      
    for n in range(maxiter):

        print(f"{calls:2} {xn:8.6f},{f:8.6f},{p:8.6f},{f/p:8.6f}")

        xn -= f/p

        f = func(xn,*(func_args or ()))
        p = prime(xn,*(prime_args or ()))

        calls += 1

        if abs(f) < tol:
            print('Found solution after',calls,'iterations!')
            return xn
    else:
        print('Exceeded maximum iterations. No solution found.')

    info = f"Converged after {calls} calls."

    return xn,f

In [51]:
def f(x):
    return x**2-4*np.sin(x)

In [52]:
def p(x):
    return 2*x-4*np.cos(x)

In [54]:
newton(f,p,3.,1e-6,100)

 1 3.000000,8.435520,9.959970,0.846942
 2 2.153058,1.294773,6.505772,0.199019
 3 1.954039,0.108439,5.403795,0.020067
 4 1.933972,0.001152,5.288920,0.000218
Found solution after 5 iterations!


np.float64(1.933753788557627)

## Secant Method

In [4]:
class secant():

    def __init__(self,func,x0,x1,tol=1e-5,args=None):

        self.x0 = x0
        self.x1 = x1

        self.tol = tol

        self.calls = 0

        if args is None:
            args = []

        self._iterate(func,args)

    def _iterate(self,func,args):

        x0,x1 = self.x0,self.x1

        y0 = func(x0,*args)

        self.calls += 1

        # print(f"{self.calls:2} {x0:8.6f},{y0:8.6f}")

        y1 = func(x1,*args)

        self.calls += 1

        d1 = (y1-y0)/(x1-x0)

        while abs(y1/d1)>self.tol:

            # print(f"{self.calls:2} {x1:8.6f},{y1:8.6f},{y1/d1:8.6f}")

            x2 = x1-y1/d1

            y2 = func(x2,*args)

            self.calls += 1

            x0,y0 = x1,y1
            x1,y1 = x2,y2

            d1 = (y1-y0)/(x1-x0)

        self.value = x1

        self.minima = y1

In [55]:
def t1(x):
    return x**2-2*x+1

def t2(x):
    return 2*x-2

In [56]:
def objective1(x):
    return x**3-x-2

def objective3(x):
    return x**2-4*x+4

In [ ]:
optimize.root_scalar(f,method='bisect',bracket=[0,2])